<a href="https://colab.research.google.com/github/harshitjain231996/EVA-4/blob/master/s9_quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from datetime import datetime

In [0]:
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-03-17 19:51:15.571662


In [0]:
!pip install albumentations==0.4.5

     |████████████████████████████████| 122kB 2.7MB/s 
     |████████████████████████████████| 634kB 8.5MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=64378 sha256=6af86f73e4b3bf7cc0560138a7dd6d84a1a8a72bd075220df4f72019aa44836e
  Stored in directory: /root/.cache/pip/wheels/f0/a0/61/e50f93165a5ec7e7f5d65064e513239505bc4c06d2289557d3
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=6598861ca3ad55838aad19d1695b7ca2396e812266fcffe7698df455acbe841b
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [0]:
!pip install cupy
from torchsummary import summary
from cuda import initialize_cuda
from data.dataset import CIFAR10
from model import QuizDNN
from models.utils.loss import cross_entropy_loss
from models.utils.optimizers import sgd_optimizer
from models.utils.callbacks import lr_scheduler
from train import train
from evaluate import val

  Using cached https://files.pythonhosted.org/packages/1f/48/bfd7252c856370e7b9e997d32fa992092fdc7014cab7fa04f5d6cd206a05/cupy-7.2.0.tar.gz


In [0]:
class Args:
    train_batch_size = 64   
    val_batch_size = 64   
    num_workers = 4   
    random_seed = 1   
    epochs = 10   
    learning_rate = 0.01   
    momentum = 0.9   
    sample_count = 10

In [0]:
cuda, device = initialize_cuda(Args.random_seed)


In [0]:
dataset = CIFAR10(
    train_batch_size=Args.train_batch_size,
    val_batch_size=Args.val_batch_size,
    cuda=cuda,
    num_workers=Args.num_workers)

In [0]:
train_loader = dataset.loader(train=True)
val_loader = dataset.loader(train=False)

In [0]:
model = QuizDNN().to(device)
summary(model, dataset.image_size)

In [0]:
losses = []
accuracies = []
correct_samples = []
incorrect_samples = []

criterion = cross_entropy_loss()  # Create loss function
optimizer = sgd_optimizer(model, Args.learning_rate, Args.momentum)  # Create optimizer
last_epoch = False

for epoch in range(1, Args.epochs + 1):
    print(f'Epoch {epoch}:')
    if epoch == Args.epochs:
        last_epoch = True
    train(model, train_loader, device, optimizer, criterion)
    val(
        model, val_loader, device, criterion, losses, accuracies,
        correct_samples, incorrect_samples, Args.sample_count, last_epoch=last_epoch
    )